In [ ]:
# import subprocess

# # 필요한 패키지 리스트
# packages = [
#     "sentence-transformers",
#     "scikit-learn",
#     "langchain",
#     "langchain-community",
#     "langchain sentence-transformers",
# ]

# # 패키지 설치
# for package in packages:
#     subprocess.run(["pip", "install", package])

In [ ]:
# # %pip install langchain lanchain-openai langchain-chroma langchain-huggingface langchain-community 
# %pip install chromadb

In [ ]:
import json
import pandas as pd
import numpy as np
import logging
import os
from sklearn.preprocessing import MultiLabelBinarizer, OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from langchain.schema import Document  # LangChain의 Document 객체 활용
from langchain_chroma import Chroma
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_openai import OpenAIEmbeddings
from langchain_community.chat_models import ChatOpenAI

from dotenv import load_dotenv
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

load_dotenv()

# 1. genres → sub_genres 매핑 딕셔너리 정의
genre_to_sub_genre = {
    # "romance": ["로맨스", "BL"],
    # "rofan": ["로판", "로맨스 판타지"],
    # "drama": ["드라마", "라이트노벨"],
    "historical": ["무협/사극", "무협", "액션/무협"],
    # "cartoon": ["코믹/일상", "일상", "개그", "감성"],
    # "horror": ["공포/스릴러", "스릴러", "미스터리"],
    # "action": ["액션", "스포츠"],
    # "fantasy": ["판타지 드라마", "판타지", "현판", "학원/판타지"],
}

# 2. JSON 데이터 로드
json_file_paths = [
    # "webtoon_kko무협.json",
    # "webnovel_kkopage_martial_arts.json",
    # "webnovel_naverseries_wuxia.json",
    # "webtoon_kkopage_historical.json",
    # "webtoon_kkopage_액션_무협.json",
    # "webtoon_naver_무협_사극.json",
]

# JSON 데이터 로드
dataset = []  # 데이터를 저장할 리스트
for file_path in json_file_paths:
    with open(file_path, "r", encoding="utf-8") as f:
        data = json.load(f)
        if isinstance(data, list):  # 리스트라면 dataset에 확장하여 추가
            dataset.extend(data)
        else:  # 단일 객체라면 그냥 추가
            dataset.append(data)

logging.info(f"총 {len(dataset)}개의 항목 로드 완료!")

# 3. 데이터 변환
documents = []

for idx, content in enumerate(dataset):
    # 키워드 리스트를 문자열로 변환
    if isinstance(content.get("keywords", ""), list):
        content["keywords"] = ", ".join(content["keywords"])

    title = content.get("title", "")
    synopsis = content.get("synopsis", "")
    keywords = content.get("keywords", "")

    # 키워드가 비어있다면, 장르를 키워드로 대체
    if not keywords:
        keywords = content.get("genre", "") # 키워드 "-" 비어있는채로 임베딩되서, 결측치 전처리를 무협으로 해버림
    genre = content.get("genre", "")

    # page_content에 텍스트 데이터 머지
    merged_text = f"제목: {title}\n장르: {genre}\n줄거리: {synopsis}\n작가: {content['author']}\n썸네일: {content['thumbnail']}\n작품_가격: {content['price']}\n링크: {content['url']}"

    # metadata 정리
    metadata = {
        "type": content.get("type", ""),
        "platform": content.get("platform", ""),
        "title": title,
        "genre": genre,
        "synopsis": synopsis,
        "keyword": keywords,
        "score": content.get("score", 0),
    }

    # Document 객체 생성
    documents.append(Document(page_content=merged_text, metadata=metadata))

logging.info(f"총 {len(documents)}개의 Document 변환 완료!")


2025-03-06 17:04:00,176 - INFO - 총 685개의 항목 로드 완료!
2025-03-06 17:04:00,182 - INFO - 총 685개의 Document 변환 완료!


#### 웹툰 임베딩

In [ ]:
# 벡터 스토어 설정
COLLECTION_NAME = "webtoon_historical"
PERSIST_DIRECTORY = "vector_store/historical/webtoon_historical"

embedding_model_name = "BAAI/bge-m3"
embedding_model = HuggingFaceEmbeddings(model_name=embedding_model_name)

# Chroma 데이터베이스에 저장
vector_store = Chroma.from_documents(
    documents=documents,
    embedding=embedding_model,
    collection_name=COLLECTION_NAME,
    persist_directory=PERSIST_DIRECTORY
)

2025-03-06 17:09:14,418 - INFO - PyTorch version 2.5.1 available.
2025-03-06 17:09:18,501 - INFO - Load pretrained SentenceTransformer: BAAI/bge-m3


#### 웹노벨 임베딩

In [ ]:
# 벡터 스토어 설정
COLLECTION_NAME = "webnovel_historical"
PERSIST_DIRECTORY = "vector_store/historical/webnovel_historical"

embedding_model_name = "BAAI/bge-m3"
embedding_model = HuggingFaceEmbeddings(model_name=embedding_model_name)

# Chroma 데이터베이스에 저장
vector_store = Chroma.from_documents(
    documents=documents,
    embedding=embedding_model,
    collection_name=COLLECTION_NAME,
    persist_directory=PERSIST_DIRECTORY
)